In [24]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport
import sweetviz as sv
from scipy.stats import chi2_contingency
import numpy as np

In [2]:
unnecessary_columns = ['V_ID','P_ID','C_YEAR','P_USER','P_PSN','V_YEAR','P_ISEV']

In [3]:
data_raw = pd.read_csv('./data/NCDB_1999_to_2014.csv',low_memory=False)

In [4]:
data_raw

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860400,2014,UU,U,UU,2,UU,UU,01,U,U,...,13,07,UUUU,01,M,24,11,1,NN,1
5860401,2014,UU,U,23,2,01,06,05,1,1,...,01,14,2006,01,M,29,96,2,09,5
5860402,2014,UU,U,14,2,01,02,01,1,5,...,01,01,2006,01,F,UU,11,2,01,1
5860403,2014,UU,U,22,1,01,06,01,2,4,...,01,22,UUUU,01,M,67,12,3,01,U


In [5]:
crashes_by_driver_df = data_raw.copy()

In [6]:
#Agregar columna con cantidad de personas por accidente
collision_columns = [col for col in crashes_by_driver_df.columns if col[0]=="C"]
#group_by_columns = collision_columns[:]
#group_by_columns.append('P_ID')
#collision_people_count_df = data[group_by_columns].groupby(group_by_columns).size().reset_index().drop('P_ID',axis=1).rename(columns={0:"A_PERS"}).drop_duplicates()
collision_people_count_df = crashes_by_driver_df[collision_columns+['P_ID']].drop_duplicates().groupby(collision_columns).size().reset_index().rename(columns={0:"C_PERS"}).drop_duplicates()

In [7]:
collision_people_count_df

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,C_PERS
0,1999,01,1,00,2,01,01,01,1,4,1,18,1
1,1999,01,1,00,2,01,01,01,1,5,1,18,1
2,1999,01,1,00,2,01,01,01,2,1,1,18,1
3,1999,01,1,00,2,01,01,01,6,3,2,18,1
4,1999,01,1,00,2,01,01,UU,1,2,3,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850390,2014,12,7,UU,2,02,UU,02,U,U,1,03,1
1850391,2014,UU,U,14,2,01,02,01,1,5,4,18,1
1850392,2014,UU,U,22,1,01,06,01,2,4,Q,18,2
1850393,2014,UU,U,23,2,01,06,05,1,1,U,18,1


In [8]:
#quedarse solo con los drivers
crashes_by_driver_df = crashes_by_driver_df[crashes_by_driver_df['P_PSN']=="11"]
crashes_by_driver_df

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
5,1999,01,1,17,2,03,QQ,QQ,1,2,...,01,01,1984,01,M,28,11,1,UU,1
6,1999,01,1,17,2,03,QQ,QQ,1,2,...,02,01,1991,01,M,21,11,1,UU,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860397,2014,UU,U,UU,2,UU,UU,01,U,U,...,10,01,UUUU,01,F,79,11,1,NN,1
5860398,2014,UU,U,UU,2,UU,UU,01,U,U,...,11,01,UUUU,01,F,20,11,1,NN,1
5860399,2014,UU,U,UU,2,UU,UU,01,U,U,...,12,01,UUUU,01,F,47,11,1,NN,1
5860400,2014,UU,U,UU,2,UU,UU,01,U,U,...,13,07,UUUU,01,M,24,11,1,NN,1


In [9]:
crashes_by_driver_df = crashes_by_driver_df.merge(collision_people_count_df,on=collision_columns,how='left')

In [10]:
print(f'Numero de registros: {crashes_by_driver_df.shape[0]}')
print(f'Numero de columnas: {crashes_by_driver_df.shape[1]}')

Numero de registros: 3926086
Numero de columnas: 23


In [11]:
crashes_by_driver_df.dtypes.to_dict()

{'C_YEAR': dtype('int64'),
 'C_MNTH': dtype('O'),
 'C_WDAY': dtype('O'),
 'C_HOUR': dtype('O'),
 'C_SEV': dtype('int64'),
 'C_VEHS': dtype('O'),
 'C_CONF': dtype('O'),
 'C_RCFG': dtype('O'),
 'C_WTHR': dtype('O'),
 'C_RSUR': dtype('O'),
 'C_RALN': dtype('O'),
 'C_TRAF': dtype('O'),
 'V_ID': dtype('O'),
 'V_TYPE': dtype('O'),
 'V_YEAR': dtype('O'),
 'P_ID': dtype('O'),
 'P_SEX': dtype('O'),
 'P_AGE': dtype('O'),
 'P_PSN': dtype('O'),
 'P_ISEV': dtype('O'),
 'P_SAFE': dtype('O'),
 'P_USER': dtype('O'),
 'C_PERS': dtype('int64')}

In [12]:
nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
others = {
    "P_SAFE" : "14",
    "P_PSN": "100",
    "V_TYPE": "24",
    "C_TRAF": "19",
    "C_RALN": "7",
    "C_RSUR": "10",
    "C_WTHR": "8",
    "C_RCFG": "13",
    "C_CONF": "42"
}
int_types = ["P_AGE","V_AGE","C_PERS","C_VEHS"]
categories = [col for col in data_raw.columns if col not in int_types + unnecessary_columns]
target = "C_SEV"

In [13]:
def replace_categories_nas(data,categories):
    nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
    others = {
        "P_SAFE" : "14",
        "P_PSN": "100",
        "V_TYPE": "24",
        "C_TRAF": "19",
        "C_RALN": "7",
        "C_RSUR": "10",
        "C_WTHR": "8",
        "C_RCFG": "13",
        "C_CONF": "42"
    }
    data[categories] = data[categories].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)
    for key,value in others.items():
        data[key] = data[key].replace(to_replace=r'^Q.*', value=value, regex=True)

def transform_hour(value):
    value = pd.to_numeric(value,errors='coerce')
    if value >= 0 and value<6:
        return '0'
    if value >= 6 and value<12:
        return '1'
    if value >= 12 and value<18:
        return '2'
    if value >= 18 and value<24:
        return '3'
    return pd.NA

In [14]:
replace_categories_nas(crashes_by_driver_df,categories)
crashes_by_driver_df['C_HOUR'] = crashes_by_driver_df['C_HOUR'].apply(transform_hour) 


In [15]:
#agregar columna con antiguedad del coche
crashes_by_driver_df['V_AGE'] = crashes_by_driver_df['C_YEAR'] - pd.to_numeric(crashes_by_driver_df['V_YEAR'], errors='coerce')

In [16]:
crashes_by_driver_df[int_types] = crashes_by_driver_df[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')

In [17]:
crashes_by_driver_df = crashes_by_driver_df.drop(unnecessary_columns,1).drop_duplicates()
crashes_by_driver_df

,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,P_SEX,P_AGE,P_SAFE,C_PERS,V_AGE
0,01,1,3,2,2.0,34,<NA>,1,5,3,03,06,M,41.0,<NA>,2,9.0
1,01,1,3,2,2.0,34,<NA>,1,5,3,03,01,M,19.0,<NA>,2,12.0
2,01,1,1,2,1.0,01,<NA>,5,3,6,18,01,M,46.0,<NA>,1,13.0
3,01,1,2,2,3.0,42,13,1,2,1,01,01,M,28.0,<NA>,2,15.0
4,01,1,2,2,3.0,42,13,1,2,1,01,01,M,21.0,<NA>,2,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926070,12,7,0,2,1.0,04,01,2,2,1,18,01,M,49.0,02,1,20.0
3926071,12,7,3,2,1.0,02,01,2,5,1,18,01,M,20.0,02,1,4.0
3926072,12,7,2,2,2.0,41,01,1,5,1,18,05,M,44.0,02,3,6.0
3926073,12,7,3,2,1.0,03,01,1,3,4,18,01,F,35.0,02,3,13.0


In [18]:
crashes_by_driver_df[target] = crashes_by_driver_df[target].astype('int8')
crashes_by_driver_df[target] = crashes_by_driver_df[target].replace({2:0})

In [19]:
crashes_by_driver_df.shape

(3871270, 17)

In [20]:
df_dtypes = pd.merge(crashes_by_driver_df.isnull().sum(axis = 0).sort_values().to_frame('missing_value').reset_index(),
         crashes_by_driver_df.dtypes.to_frame('feature_type').reset_index(),
         on = 'index',
         how = 'inner')
df_dtypes

,index,missing_value,feature_type
0,C_SEV,0,int64
1,C_PERS,0,int8
2,C_MNTH,142,object
3,C_VEHS,243,float64
4,C_WDAY,832,object
5,V_TYPE,30364,object
6,C_HOUR,40330,object
7,C_RSUR,52468,object
8,C_WTHR,57515,object
9,C_CONF,126362,object


In [21]:
print(f'Registros Sin muertes {crashes_by_driver_df[crashes_by_driver_df[target]==0][target].count()}')
print(f'Registros Con muertes {crashes_by_driver_df[crashes_by_driver_df[target]==1][target].count()}')

Registros Sin muertes 3814337
Registros Con muertes 56933


In [37]:
crashes_by_driver_df.to_csv('./data/crash_transformed_c_sev.csv',index=False)

In [38]:
profile = ProfileReport(crashes_by_driver_df, minimal=True)
profile.to_file("./eda_output/output_csev.html")